## Examining timing side-channel attacks

The idea is to show that time analysis can reveal meaningful information about a string used in a secret or a password if the amount of time it takes to perform the string match isn't buffered. 

The example will be to show how the libc strcmp method reveals information about a secret string and, possibly, the string itself in a brute force attack. We'll be using python code to demonstrate the action and analysis, but this can be performed with any language that uses similar methods.


Our secret string is 'apple'.

A client presents the secret string to the server, who knows the secret string, in an operation that will fail if the secret is incorrect.

So:

Client -> 'apple' -> Server  

Server: 
- receives a buffer containing the string. 
- gets a copy of the secret 
- strcmp's the two strings
    - analyze bytes from l to r
    - when a byte fails, return False
    - if no bytes fail return True
- return the outcome to the client

The problem is that for every correct chunk of text on the LHS the "fail" takes longer to return. 

So "b" or "broken" returns a failure instantly, but a, aplomb, app, application, etc will each take slightly longer for each character matched. The times are tiny, but it's possible to model the jitter windows such that these lengths don't get lost in the noise. This can drastically cut down the time of a brute force attack.

TL;DR - using sound hash functions which return in a constant time and not rolling-your-own solves this problem.

Also: for authentication purposes it's an antipattern to store the secret plaintext or decode an encrypted secret to compare in plaintext. Hashing the secret, storing the hash then comparing the hashes also obscures the size of your secret by making the compared strings the same lengh, and increasing the problem for a brute force attack.

In [13]:
# need to use attotime here to get fractional nanoseconds
import sys, os, time, string, attotime, secrets
import pandas as pd
import nump

In [2]:
# in a long string we just want to guess the first character
# mysecret = secrets.token_urlsafe()
mysecret = 'rMAwzI_52tVSvgKzRo3PJ0G7_Danw7jlaZ-6vTkOUiQ'

# we only care about urlsafe characters - alpha, numeric, dash and underscore. Omitting . and ~ (RFC 3986)
valid_chars = list(string.ascii_letters)
for x in range(0,10):
    valid_chars.append(str(x))
valid_chars.append('-')
valid_chars.append('_')

In [5]:
def compare1(str1, str2):
    """
    Here we're going to amplify the lib strcmp delays to make the weakness easier to analyze:
    1. evaluate strings l-to-r
    2. dope a positive match with a tiny wait to emphasize the delay for demonstration
    3. return a False match immediately
    """

    one, two = list(str1), list(str2)
    for x in range(0,len(one)):
        if one[x] == two[x]:
            time.sleep(.001)
            pass
        else:
            return False
    return True

### First example - guess the first letter

Here we'll just run through each letter 1k times adding completion time to an arry. Then we'll find the longest run of the letters

In [22]:
totals=[]
for loop in range(1000):
    stash = {}
    for l in valid_chars:
        mystr = l *10
        start = attotime.attodatetime.now()
        compare1(mystr, mysecret)
        end = attotime.attodatetime.now()
        duration = end - start
        stash[mystr] = str(duration)

In [23]:
# fastest
sorted(stash.items(), key=lambda x: x[1])[:10]

[('UUUUUUUUUU', '0:00:00.00021195411682129'),
 ('8888888888', '0:00:00.00021290779113769'),
 ('TTTTTTTTTT', '0:00:00.0002129077911377'),
 ('IIIIIIIIII', '0:00:00.0002131462097168'),
 ('WWWWWWWWWW', '0:00:00.000213623046875'),
 ('EEEEEEEEEE', '0:00:00.0002138614654541'),
 ('YYYYYYYYYY', '0:00:00.0002138614654541'),
 ('7777777777', '0:00:00.0002138614654541'),
 ('nnnnnnnnnn', '0:00:00.00021409988403321'),
 ('HHHHHHHHHH', '0:00:00.00021409988403321')]

In [25]:
# slowest returns - note how much slower lower case r (leading character match) is compared to the rest
sorted(stash.items(), key=lambda x: x[1])[-10:
                                         ]

[('LLLLLLLLLL', '0:00:00.00022125244140625'),
 ('aaaaaaaaaa', '0:00:00.00022602081298828'),
 ('FFFFFFFFFF', '0:00:00.00022697448730469'),
 ('ffffffffff', '0:00:00.00022721290588379'),
 ('6666666666', '0:00:00.00023198127746582'),
 ('vvvvvvvvvv', '0:00:00.00023794174194336'),
 ('uuuuuuuuuu', '0:00:00.00023818016052246'),
 ('tttttttttt', '0:00:00.00026583671569824'),
 ('ssssssssss', '0:00:00.00033998489379883'),
 ('rrrrrrrrrr', '0:00:00.00161886215209961')]

## example of brute force strcmp analysis attacks 

Here's a dictionary matcher

In [10]:
name='apple'
stash2 = {}
for c in valid_chars:
    stub = c
    for loop in range(0, len(name)):
        stublist = [stub + l for l in valid_chars]
        for s in stublist:
            start = attotime.attodatetime.now()
            if compare1(s, name):
                if s == name:
                    print('Name is', s)
                    break
                else:
                    end = attotime.attodatetime.now()
                    duration = end - start
                    stash2[s] = str(duration)
                    print(s)
                    stub = s
                    break
            else:
                end = attotime.attodatetime.now()
                duration = end - start
                stash2[s] = str(duration)
                pass

ap
app
appl
Name is apple
Name is apple


In [11]:
sorted(stash2.items(), key=lambda x: x[1])[-10:]

[('appj', '0:00:00.00432777404785156'),
 ('appe', '0:00:00.00439119338989258'),
 ('app', '0:00:00.0044102668762207'),
 ('appi', '0:00:00.00441098213195801'),
 ('appg', '0:00:00.00449085235595703'),
 ('applc', '0:00:00.00566577911376953'),
 ('appla', '0:00:00.00567793846130371'),
 ('applb', '0:00:00.00574898719787598'),
 ('appl', '0:00:00.00579214096069336'),
 ('appld', '0:00:00.00586605072021484')]

In [14]:
and a modified dictionary word

SyntaxError: invalid syntax (3740785661.py, line 1)

In [12]:
name='orang00t4_n'
stash3 = {}
for c in valid_chars:
    stub = c
    for loop in range(0, len(name)):
        stublist = [stub + l for l in valid_chars]
        for s in stublist:
            start = attotime.attodatetime.now()
            if compare1(s, name):
                if s == name:
                    print('Name is', s)
                    break
                else:
                    end = attotime.attodatetime.now()
                    duration = end - start
                    stash3[s] = str(duration)
                    print(s)
                    stub = s
                    break
            else:
                end = attotime.attodatetime.now()
                duration = end - start
                stash3[s] = str(duration)
                pass
            
sorted(stash3.items(), key=lambda x: x[1])[-10:]

or
ora
oran
orang
orang0
orang00
orang00t
orang00t4
orang00t4_
Name is orang00t4_n
Name is orang00t4_n


[('orang00t4w', '0:00:00.01302409172058106'),
 ('orang00t4_g', '0:00:00.01305103302001953'),
 ('orang00t4_f', '0:00:00.01306295394897461'),
 ('orang00t4y', '0:00:00.01307415962219238'),
 ('orang00t4_d', '0:00:00.01309800148010254'),
 ('orang00t4_m', '0:00:00.01314830780029296'),
 ('orang00t4_h', '0:00:00.01318287849426269'),
 ('orang00t4_j', '0:00:00.01326775550842285'),
 ('orang00t4_i', '0:00:00.0136568546295166'),
 ('orang00t4_', '0:00:00.01391315460205078')]